In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import corner
import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook
from collections import OrderedDict
from mosfit.plotting import bandcolorf

with open('../products/walkers.json', 'r') as f:
    data = json.loads(f.read())

photo = data['photometry']
model = data['models'][0]

real_data = len([x for x in photo if 'realization' not in x]) > 0

band_attr = ['band', 'instrument', 'system', 'bandset']
band_list = list(set([tuple(x.get(y, '')
                            for y in band_attr) for x in photo]))
real_band_list = list(set([tuple(x.get(y, '')
                                 for y in band_attr) for x in photo
                                 if 'realization' not in x]))

fig = plt.figure(figsize=(12,6))
used_bands = []
for full_band in tqdm_notebook(band_list):
    (band, inst, syst, bset) = full_band
    extra_nice = ', '.join(list(filter(None, OrderedDict.fromkeys((inst, syst, bset)).keys())))
    nice_name = band + (' [' + extra_nice + ']') if extra_nice else ''
    for wi in range(1, len(model['realizations'])):
        vec = [
            (x['time'], x['magnitude']) for x in photo
            if x.get('realization', None) == str(wi) and
            tuple(x.get(y, '') for y in band_attr) == full_band]
        if not len(vec):
            continue
        xs, ys = zip(*vec)
        label = '' if full_band in used_bands or full_band in real_band_list else nice_name
        plt.plot(xs, ys, c=bandcolorf(band), label=label, linewidth=0.5)
        if label:
            used_bands = list(set(used_bands + [full_band]))
    if real_data:
        for s in range(2):
            if s == 0:
                cond = True
                symb = 'v'
            else:
                cond = False
                symb = 'o'
            vec = [(float(x['time']), float(x['magnitude']), float(x.get('e_magnitude', 0.0))) for x in photo
                   if 'realization' not in x and
                   x.get('upperlimit', False) == cond and
                   tuple(x.get(y, '') for y in band_attr) == full_band]
            if not len(vec):
                continue
            xs, ys, yes = zip(*vec)
            label = nice_name if full_band not in used_bands else ''
            plt.errorbar(xs, ys, yerr=yes, color=bandcolorf(band), fmt=symb,
                         label=label,
                         markeredgecolor='black', markeredgewidth=1, capsize=5,
                         elinewidth=2, capthick=2, zorder=10)
            plt.errorbar(xs, ys, yerr=yes, color='k', fmt=symb, capsize=6,
                         elinewidth=3, capthick=3, zorder=5)
            if label:
                used_bands = list(set(used_bands + [full_band]))
plt.gca().invert_yaxis()
plt.margins(0.1,0.1)
plt.legend()
plt.show()
fig.savefig('../products/lc.pdf')

# Construct walker arrays for corner
print('Preparing corner plot, takes some time.')
corner_input = []
for realization in model['realizations']:
    pars = realization['parameters']
    var_names = ['$' + ('\\log\\, ' if pars[x]['log'] else '') +
                 pars[x]['latex'] + '$' for x in pars]
    corner_input.append([np.log10(pars[x]['value']) if
                         pars[x]['log'] else pars[x]['value'] for x in pars])
cfig = corner.corner(corner_input, labels=var_names)
cfig.savefig('../products/corner.pdf')